<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow import keras
import json
import spacy
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,Flatten
from keras .optimizers import SGD
import random

In [ ]:
data_file=open("intents.json").read()
intents=json.loads(data_file)

In [ ]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [ ]:
nlp=spacy.load("en_core_web_sm")
words =[]
classes=[]
documents=[]
ignore_words=['?','!']

In [ ]:
words=[]
documents=[]
classes=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        tokens= nlp(pattern)
        l=[]
        for i in tokens:
             i=str(i)
             l.append(i)
        words.extend(l)
        #add documents in the corpus
        documents.append((l, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
print(l)
print(words)
print(len(words))
print(documents)
print(classes)
len(classes)

['Looking', 'up', 'hospital', 'details']
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'do', 'nt', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'e

9

In [ ]:
new_words=[]
for i in words:
    new_words.append(i.lower())
print(words)
print(new_words)

unique_words = set(new_words)
unique_words
word2int = {}
int2word = {}
vocab_size = len(unique_words)
print(vocab_size)
for i,word in enumerate(unique_words):
  word2int[word] = i
  int2word[i] = word 
print(word2int)
print(int2word)
sentences=list()



for i in documents:
    sentences.append(i[0])
print(sentences)    
sentences2=[]
for i in sentences:
   sentences1=[]
   for j in i:
      j=j.lower()
      sentences1.append(j)   
   sentences2.append(sentences1)
print(sentences2)      

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'do', 'nt', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'entry', 'I', 'want', 'to', 'log', 'blood',

In [ ]:
window_size = 2
data = []
for sentence in sentences2 :
  for wordId,word in enumerate(sentence):
    for nxword in sentence[max(wordId - window_size,0):min(wordId + window_size, len(sentence))+1]:
        if nxword != word:
           data.append([word,nxword])

data

In [ ]:
def Encoder(datapointindex, vocab_size):
  temp = np.zeros(vocab_size)
  temp[datapointindex] = 1
  return temp

In [ ]:
train_x = []
train_y = []

for worddata in data:
  try:
    train_x.append(Encoder(word2int[worddata[0]], vocab_size))
    train_y.append(Encoder(word2int[worddata[1]], vocab_size))
  except:

    continue

In [ ]:
len(train_x[0])


92

In [ ]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),) activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(train_x,train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist)


print("model created")

In [ ]:
|